In this notebook I investigate the effect of inequality reduction on GHG emissions

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from kde import *
from perc import *
%matplotlib inline

from wb_api_wrapper import *


pd.set_option('display.max_colwidth', 200)
pd.set_option('display.width', 200)
pd.set_option('display.precision', 2)
pd.set_option('display.max_rows', 500)


C:\Users\Admin\Anaconda3\lib\site-packages\pandas\io\wb.py:19: FutureWarning: 
The pandas.io.wb module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)


In [2]:
datadir = "csvdata/";

In [3]:
any_name_to_wb_name = pd.read_csv("any_name_to_wb_name.csv",index_col="any",squeeze=True)
iso_to_name = pd.read_csv("iso3_to_wb_name.csv",usecols=["iso3","country"],index_col="iso3",squeeze=True)
names_to_iso = pd.read_csv("names_to_iso.csv",usecols=["country","iso3"],index_col="country",squeeze=True)

### Data from World bank API

In [4]:
wb_pop = get_wb_mrv("SP.POP.TOTL","wb_pop")
wb_emis = get_wb_mrv("EN.ATM.CO2E.FF.KT","wb_emis") #CO2 emissions from fossil fuels, total;

### Data from Maros

In [ ]:
#personal data
persdata=pd.read_csv("csvdata/PERSDATA.csv").replace(iso_to_name)  #here emost of weights are 1 but some weights are 0.6
persdata.columns = map(str.lower, persdata.columns)
persdata=persdata.set_index("schema").drop("arm04").reset_index()#drops armenia 2004 since we have armenia 2012
persdata.schema.unique() #countries should apper only once

In [ ]:
#household data
hhdata = pd.read_csv("csvdata/HHLDDATA.csv").replace(iso_to_name)
hhdata.columns = map(str.lower, hhdata.columns)
hhdata = hhdata.set_index("schema").drop("arm04").reset_index()#drops armenia 2004 since we have armenia 2012
hhdata.schema.unique()#countries should apper only once

In [ ]:
persdata.head()

In [ ]:
#emissions per household and per capita
emis_hh = pd.read_csv("csvdata/emissionsbyhousehold.csv").replace(iso_to_name).rename(columns={"value": "emis_hh"})
emis_hh.schema.unique() #countries should apper only once (Armenia  is only once in emissionsbyhousehold)

In [ ]:
weight_hh = hhdata.set_index(['country',"hid"])["weight"]

In [ ]:
#number of persons per household
nb_people_in_hh =persdata.groupby(["country","hid"])["pid"].count()
nb_people_in_hh.name = "nb_people_in_hh"
nb_people_in_hh.head();
#total number of people in the economy
nb_people_tot = (weight_hh*nb_people_in_hh).sum(level="country")
(nb_people_tot/wb_pop).dropna()

Previous cell confirms how I should compute population from HHdata

In [ ]:
#emissions per household and per capita
expend = pd.read_csv("csvdata/PerCapitaExpenditures.csv").replace(iso_to_name).rename(columns={"value": "expend"})
expend.schema.unique()

In [ ]:
df["expend"] = expend.set_index(["country","hid"]).expend

In [ ]:
df.ix["Vietnam"].plot(x="expend",y="emis_hh",kind="scatter")

### Gtap

In [ ]:
gtap_to_wb_name = pd.read_csv("gtap_to_wb_name.csv",index_col="gtap_region",squeeze=True)
gtap_to_wb_name;

In [ ]:
gtap = pd.read_csv("sec_data_gtap.csv").replace(gtap_to_wb_name)
gtap["country"]=gtap.gtap_region.str.upper().str.strip().replace(gtap_to_wb_name)
gtap =gtap.set_index(["country","cpc2"]).drop(["gtap_region"],axis=1)

In [ ]:
gtap.conso_usd.sum(level="country");

NY.GDP.MKTP.CD 	GDP (current US$)  #mean .55, not that bad
NY.GDP.MKTP.CN 	GDP (current LCU)
NY.GDP.MKTP.KD 	GDP (constant 2005 US$)
NY.GDP.MKTP.KN 	GDP (constant LCU)
NY.GDP.MKTP.PP.CD 	GDP, PPP (current international $)
NY.GDP.MKTP.PP.KD 	GDP, PPP (constant 2011 international $)

In [ ]:
((expend.set_index(["country","hid"]).expend * nb_people_in_hh ).sum(level="country") / gtap.conso_usd.sum(level="country")).dropna()

In [ ]:
(1e6* gtap.conso_usd.sum(level="country") / get_wb_mrv("NY.GDP.MKTP.KN","foo")).dropna();

In [ ]:
((gtap.conso_usd*gtap.emissions_g_usd).sum(level="country")/wb_emis).dropna()

###df

In [ ]:
#checks sanity with the index
emis_hh[['country',"hid"]].duplicated().sum()  , hhdata[['country',"hid"]].duplicated().sum()

In [ ]:
df= pd.concat([nb_people_in_hh,weight_hh,emis_hh.set_index(["country","hid"])["emis_hh"]],axis=1)

In [ ]:
df.columns

In [ ]:
(df[[ 'weight', 'emis_hh']].prod(axis=1).sum(level="country")/(1e6*wb_emis)).dropna()

In [ ]:
df[[ 'weight', 'emis_hh']].prod(axis=1).sum(level="country")/1e6

In [ ]:
(df[[ 'weight', 'emis_hh']].prod(axis=1).sum(level="country"))

In [ ]:
(df[['nb_people_in_hh',  'emis_hh']].prod(axis=1).sum(level="country")/wb_emis).dropna()

In [ ]:
df.columns

In [ ]:
((df.emis_hh/(df.weight*df.nb_people_in_hh)).sum(level="country")/wb_emis).dropna()

In [ ]:
(df[['nb_people_in_hh', 'weight', 'emis_hh']].prod(axis=1).sum(level="country")/wb_emis).dropna()

###Data from GTAP and computation in SQL

In [ ]:
gtap_emis = pd.read_csv("tot_emis_gtap.csv",header=None,names=["country","emis"],index_col = "country", squeeze=True)

In [ ]:
gtap_emis;

In [ ]:
(gtap_emis / wb_emis).dropna()

In [ ]:
(gta{ap_emis / tot_emis_maybe).dropna()

In [ ]:
wb_emis.ix[wb_emis.index]

In [ ]:
wb_emis = wb_emis.ix[tot_emis.index]

In [ ]:
tot_emis

###Data from IEA

In [ ]:

iea_emis=pd.read_csv("carbon_emissions_from_iea.csv",skiprows=[1],na_values=[".."]).dropna(axis=1,how="all").dropna().replace(any_name_to_wb_name).set_index("country")
iea_emis = iea_emis.ix[wb_emis.index].astype("float")

In [ ]:
iea_emis

In [ ]:
wb_emis

In [ ]:
iea_emis.iloc[:,-1]

###Comparing various sources

In [ ]:
(tot_pop/wb_pop).dropna()

In [ ]:
wb_emis_pc=wb_emis_pc.ix[tot_emis.index]
wb_emis = wb_emis.ix[tot_emis.index] #kilotons, for some reason
wb_emis;

In [ ]:
tot_emis/wb_emis

In [ ]:
(tot_emis/tot_pop)/(wb_emis_pc)

In [ ]:
#expenditures per capita
hh_exp = pd.read_csv("csvdata/PerCapitaExpenditures.csv").replace(iso_to_name)
hh_exp.rename(columns={"value": "hh_exp_pc"},inplace=True)
hh_exp.set_index(["country","hid"],inplace="True")

In [ ]:
#builds dataframe indexed by country and hid with all the data
df = pd.DataFrame(hh_exp["hh_exp_pc"])
df["hh_emi"]=hh_emi["hh_emis"]
df["hh_emi_pc"]=hh_emi_pc
df["nb_people"]=nb_people
df["w"]=hop["weight"]
df["hh_emis_perdol"]=df["hh_emi_pc"]/df["hh_exp_pc"]

#Vietnam 
vn=df.ix["VNM",:]

In [ ]:
def wavg(val_col_name, wt_col_name):
    """computes the average of val_col_name weighted by wt_col_name"""
    def inner(group):
        return (group[val_col_name] * group[wt_col_name]).sum() / group[wt_col_name].sum()
    #inner.__name__ = res_name
    return inner

In [ ]:
flat_df =  df.reset_index()
country_list = flat_df.country.unique()

In [ ]:
#Categories USD per day categories
income_bins = np.array([0,1,1.25,2,2.5,4]+[10*2.**i for i in range(0,15,1)]) 
income_cut = pd.cut(flat_df["hh_exp_pc"]/365, income_bins)

income_cut.unique()


In [ ]:
flat_df

In [ ]:
#
stats = pd.DataFrame()
grouped =flat_df.groupby(["country",income_cut])
stats["avg_emis_pc"]=grouped.apply(wavg('hh_emi_pc', 'w'))
stats["nb_people0"] =grouped["nb_people"].sum()
#stats.sort_index(inplace=True)
#stats["nb_people1"]=grouped["nb_people"]
stats

In [ ]:
#stats.ix[c]
#country_stats

In [ ]:
#for c in country_list:
#def lift_folks(country_stats):
def lift_folks(country_stats):
    #out = pd.DataFrame(index=country_stats.index)
    emis =pd.DataFrame(index=["emis"])
    out= country_stats.ix[:,["nb_people0"]]
    for i in range(1,10):
        nam = 'min'+str(income_bins[i])
        out[nam] = country_stats["nb_people0"]
        out.ix[i,nam] = country_stats.ix[0:i+1,"nb_people0"].sum()
        out.ix[0:i,nam] =0
        #emis[str(i)]= np.average(country_stats["avg_emis_pc"],weights=out[nam])
    return out
 #   stats.ix[c,[out.columns.values]]=out

In [ ]:
emis_sce_country = pd.DataFrame()
#computes new emissions at once
for c in country_list:
    emis_sce_country[c]= stats.ix[c]["avg_emis_pc"].dot(lift_folks(stats.ix[c,:]))/(stats.ix[c]["nb_people0"].sum())
    
emis_sce_country    

In [ ]:
#emis_sce_country['tot']=emis_sce_country.sum(axis=1)  

In [ ]:
(emis_sce_country/emis_sce_country.ix["nb_people0",:]-1)*100

In [ ]:
#paf = stats.reset_index().groupby("country").apply(lift_folks)
#paf.reset_index().groupby("country").apply(wavg("avg_emis_pc","nb_people1"))
#paf

In [ ]:
#pd.concat([flat_df,pd.get_dummies(income_cut)],axis=1).set_index(["country","hid"])
income_cut = pd.cut(df["hh_exp_pc"]/365, income_bins)

income_dummies= pd.get_dummies(income_cut)
df= pd.concat([df,income_dummies],axis=1)
df

In [ ]:
stats.ix[:,"nb_people1"] = stats.ix[:,"nb_people"]
stats.ix[:,"nb_people1"]

In [ ]:
stats.ix[pd.MultiIndex("ALB)]

In [ ]:
wp(vn["hh_exp_pc"],vn["nb_people"],np.arange(0,1,0.1))

In [ ]:
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 15}
plt.rc('font', **font)
plt.figure(figsize=(9,7))
plt.plot((df["hh_exp_pc"]),df["hh_emi_pc"]/df["hh_exp_pc"],color="blue",marker='.',markeredgewidth=1,linestyle='None',markersize=12,clip_on=False)
plt.xlabel("$\$/yr$")
plt.ylabel("$gCO_2/\$$")
plt.xscale("log")

In [ ]:
def weighted_rolling_mean(x,y,w,kernel):
    

In [ ]:
from statsmodels.nonparametric.kernel_regression import KernelReg
import numpy as np
import matplotlib.pyplot as plt
x = np.linspace(0,2*np.pi,100)
y = np.sin(x) + np.random.random(100) * 0.2
kr = KernelReg(y,x,'o')
plt.plot(x, y, '+')
y_pred, y_std = kr.fit(x)
plt.plot(x, y_pred)

In [ ]:
def plot_distribution(y,w,thecolor="red"):
    hist, bin_edges = np.histogram(y,bins=100,weights=w,density=True)
    width = 0.5 * (bin_edges[1] - bin_edges[0])
    center = (bin_edges[:-1] + bin_edges[1:]) / 2
    xs = np.linspace(center.min(),center.max(),300)
    plt.bar(center, hist, align='center', width=width,alpha=0.0,color=thecolor)
    density = gaussian_kde(y,weights=w)
    density._compute_covariance()
    plt.plot(xs,density(xs),linewidth=2,color=thecolor)


In [ ]:
plot_distribution(np.log(vn.hh_emi_pc),vn.w)

